In [1]:
import pandas as pd

df_data = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\주제1-20230804T161737Z-001\\주제1\\data.csv", encoding="utf-8")
df_submission = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\주제1-20230804T161737Z-001\\주제1\\submission.csv", encoding="utf-8")
df_target = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\주제1-20230804T161737Z-001\\주제1\\target.csv", encoding="utf-8")
df_population = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\주제1-20230804T161737Z-001\\주제1\\인천_학령인구_추계.csv", encoding ="euc-kr")
df_distance = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\8.7 데이터 분석\\df_distance.csv", encoding = "utf-8")
df_bus = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\8.7 데이터 분석\\인천광역시_중구동구_버스정류장_위도경도.csv", encoding = "utf-8")
df_warning = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\8.7 데이터 분석\\인천유흥_위경도.csv", encoding = "cp949")
df_factory = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\8.7 데이터 분석\\[안전폴더]공장,경찰서,소방서,의료기관\\searchData\\인천광역시_공장등록 현황_중구,동구.csv", encoding = 'cp949' )
df_hospital = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\8.7 데이터 분석\\[안전폴더]공장,경찰서,소방서,의료기관\\searchData\\인천광역시_의료기관 현황_중구,동구.csv", encoding = "cp949")
df_police1 = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\8.7 데이터 분석\\[안전폴더]공장,경찰서,소방서,의료기관\\searchData\\Police\\경찰청_경찰관서 위치 주소 현황_중구,동구.csv", encoding = "cp949")
df_police2 = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\8.7 데이터 분석\\[안전폴더]공장,경찰서,소방서,의료기관\\searchData\\Police\\전국경찰관서안내.csv", encoding = "cp949")
df_fire1 = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\8.7 데이터 분석\\[안전폴더]공장,경찰서,소방서,의료기관\\searchData\\FireStation\\전국 119안전센터 현황(2021.12.31)_중구,동구.csv", encoding = "cp949")
df_fire2 = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\8.7 데이터 분석\\[안전폴더]공장,경찰서,소방서,의료기관\\searchData\\FireStation\\전국 소방서 현황_중구,동구.csv", encoding = "cp949")
df_distance_weight = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\최종_distance_weight.csv", encoding = "utf-8")

In [2]:
import itertools
import math
    
values_to_remove = ['F1', 'F4', 'F7', 'F11', 'F15', 'F19', 'F5', 'F39', 'F9', 'F18', 'F20', 'F23', 'F25', 'F45']
df_target = df_target[~df_target['학교 id'].isin(values_to_remove)]

df_target = df_target.reset_index(drop=True)  # 인덱스 초기화

df_distance_weight = df_distance_weight[~df_distance_weight['학교 id'].isin(values_to_remove)]
df_distance_weight = df_distance_weight.reset_index(drop=True) 


def calculate_distance(lat1, lon1, lat2, lon2):
    """
    두 지점의 위도와 경도로부터 거리를 구하는 함수 (단위: km)
    
    :param lat1: 지점 1의 위도
    :param lon1: 지점 1의 경도
    :param lat2: 지점 2의 위도
    :param lon2: 지점 2의 경도
    :return: 두 지점 사이의 거리 (단위: km)
    """
    # 지구의 반지름 (단위: km)
    earth_radius = 6371.0

    # 라디안으로 변환
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine 공식 적용
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = earth_radius * c

    return distance

candidates = []
for i, row in df_target.iterrows() :
    candidates.append((row['Y'], row['X'], 750, row['학교 id']))

demands = []
for i, row in df_data.iterrows() :
    demands.append((row['Y'], row['X'], row['초등학령인구'], row['수요지 id']))

distance_matrix = [[calculate_distance(candidate[0], candidate[1], demand[0], demand[1]) for demand in demands] for candidate in candidates]

def solve_cflp(distance_matrix, capacity, demands, df_distance_wieght):
    num_candidates = len(distance_matrix)
    num_demands = len(distance_matrix[0])
    
    assignment = [-1] * num_demands
    var = 0
    var_weight = 0
    total_weight = 0
    
    for demand_index in range(num_demands):
        min_weight = float('inf')
        best_candidate = -1
        
        for candidate_index in range(num_candidates):
            var_weight = df_distance_weight.loc[var, 'weight']
            var += 1
            if capacity[candidate_index] >= demands[demand_index][2]:
                weight = distance_matrix[candidate_index][demand_index] * var_weight
                if weight < min_weight:
                    min_weight = weight
                    best_candidate = candidate_index
        
        if best_candidate != -1:
            # Check if the total demand capacity of the candidate is within the range [240]
            if  0 <= capacity[best_candidate]-demands[demand_index][2] :
                assignment[demand_index] = best_candidate
                total_weight += weight
                capacity[best_candidate] -= demands[demand_index][2]
    
    return assignment, total_weight

# 초기 용량 정보 (용량은 수요지 수용량으로 초기화)
capacity = [candidate[2] for candidate in candidates]

# CFLP 알고리즘 실행
best_assignment, total_weight = solve_cflp(distance_matrix, capacity, demands, df_distance_weight)


# 결과 출력
#for demand_index, candidate_index in enumerate(best_assignment):
#    if candidate_index != -1:
#        demand = demands[demand_index]
#        candidate = candidates[candidate_index]
#        print(f"수요지 {demand}가 후보지 {candidate}에 할당됨")

# 결과 출력
assigned_candidates = [None] * len(candidates)



for demand_index, candidate_index in enumerate(best_assignment):
    if candidate_index != -1:
        if assigned_candidates[candidate_index] is None:
            assigned_candidates[candidate_index] = []
        assigned_candidates[candidate_index].append(demands[demand_index])

# 중복되는 수요지 확인
def check_duplicate_demands(assigned_candidates):
    seen_demands = set()
    duplicate_demands = []

    for demand_list in assigned_candidates:
        if demand_list is not None:
            for demand in demand_list:
                if demand in seen_demands:
                    duplicate_demands.append(demand)
                else:
                    seen_demands.add(demand)

    return duplicate_demands

# 후보지별 할당 결과 출력
for candidate_index, demand_list in enumerate(assigned_candidates):
    if demand_list:
        candidate = candidates[candidate_index]
        unique_demands = set(demand_list)
        demand_names = [f"수요지 {demand[3]}" for demand in unique_demands]
        demands_assigned = ', '.join(demand_names)
        print(f"후보지 {candidate[3]}에 {demands_assigned} 할당됨")
    else:
        print(f"후보지 {candidates[candidate_index][3]}에 할당된 수요지 없음")

# 중복되는 수요지 확인 결과 출력
duplicate_demands = check_duplicate_demands(assigned_candidates)
if duplicate_demands:
    duplicate_demand_names = [f"수요지 {demand[3]}" for demand in duplicate_demands]
    duplicate_names_str = ', '.join(duplicate_demand_names)
    print(f"\n중복되는 수요지: {duplicate_names_str}")
else:
    print("\n중복되는 수요지 없음")

    
    # 각 후보지당 수요지 용량 합, 개수 출력
total_demand_num = 0
for candidate_index, demand_list in enumerate(assigned_candidates):
    if demand_list:
        candidate = candidates[candidate_index]
        total_demand = sum([demand[2] for demand in demand_list])
        total_demand_num += len(set(demand_list))
        print(f"후보지 {candidate[3]}의 수요지 용량 합: {total_demand}")
    else:
        print(f"후보지 {candidates[candidate_index][3]}에 할당된 수요지 없음")
print(total_demand_num)


# df_submission 데이터프레임에 각 수요량 할당
for candidate_index, demand_list in enumerate(assigned_candidates):
    if demand_list:
        candidate_id = candidates[candidate_index][3]
        for demand in demand_list:
            demand_id = demand[3]
            demand_capacity = demand[2]

            condition = (df_submission['수요지 id'] == demand_id) & (df_submission['학교 id'] == candidate_id)
            df_submission.loc[condition, '할당 학령인구'] = demand_capacity

print("총 weight:", total_weight)

후보지 F2에 수요지 C78, 수요지 C83, 수요지 C86, 수요지 C82 할당됨
후보지 F3에 수요지 C15, 수요지 C13, 수요지 C14, 수요지 C23, 수요지 C29 할당됨
후보지 F6에 수요지 C246, 수요지 C219 할당됨
후보지 F8에 수요지 C280, 수요지 C284, 수요지 C274, 수요지 C275, 수요지 C250, 수요지 C267, 수요지 C281, 수요지 C262, 수요지 C286, 수요지 C251, 수요지 C285, 수요지 C279 할당됨
후보지 F10에 수요지 C296, 수요지 C295, 수요지 C306, 수요지 C309, 수요지 C311, 수요지 C305, 수요지 C302, 수요지 C310, 수요지 C299, 수요지 C307 할당됨
후보지 F12에 수요지 C154, 수요지 C130, 수요지 C104, 수요지 C119, 수요지 C124, 수요지 C114, 수요지 C108, 수요지 C109, 수요지 C105, 수요지 C149, 수요지 C126, 수요지 C146, 수요지 C183, 수요지 C107, 수요지 C156, 수요지 C111, 수요지 C135, 수요지 C113, 수요지 C157, 수요지 C112, 수요지 C141, 수요지 C147, 수요지 C155, 수요지 C166, 수요지 C120, 수요지 C167, 수요지 C148, 수요지 C117, 수요지 C125, 수요지 C103, 수요지 C110, 수요지 C140, 수요지 C118 할당됨
후보지 F13에 할당된 수요지 없음
후보지 F14에 수요지 C209, 수요지 C231, 수요지 C239, 수요지 C238 할당됨
후보지 F16에 수요지 C162, 수요지 C201, 수요지 C210, 수요지 C129, 수요지 C214, 수요지 C192, 수요지 C220, 수요지 C179, 수요지 C223, 수요지 C178, 수요지 C153, 수요지 C180, 수요지 C213, 수요지 C164, 수요지 C221, 수요지 C181, 수요지 C203, 수요지 C227, 수요지 C202, 수요지 C226, 

In [3]:
df_submission = pd.read_csv("C:\\Users\\류석현\\Desktop\\field\\주제1-20230804T161737Z-001\\주제1\\submission.csv", encoding="utf-8")

In [4]:
values_to_remove = ['F6', 'F13','F14','F17','F26','F27','F29','F30','F32','F37','F38','F41','F42','F43','F46','F49','F50']
df_target = df_target[~df_target['학교 id'].isin(values_to_remove)]

df_target = df_target.reset_index(drop=True)  # 인덱스 초기화

df_distance_weight = df_distance_weight[~df_distance_weight['학교 id'].isin(values_to_remove)]
df_distance_weight = df_distance_weight.reset_index(drop=True) 

candidates = []
for i, row in df_target.iterrows() :
    candidates.append((row['Y'], row['X'], 750, row['학교 id']))

demands = []
for i, row in df_data.iterrows() :
    demands.append((row['Y'], row['X'], row['초등학령인구'], row['수요지 id']))

distance_matrix = [[calculate_distance(candidate[0], candidate[1], demand[0], demand[1]) for demand in demands] for candidate in candidates]

def solve_cflp(distance_matrix, capacity, demands, df_distance_wieght):
    num_candidates = len(distance_matrix)
    num_demands = len(distance_matrix[0])
    
    assignment = [-1] * num_demands
    var = 0
    var_weight = 0
    total_weight = 0
    
    for demand_index in range(num_demands):
        min_weight = float('inf')
        best_candidate = -1
        
        for candidate_index in range(num_candidates):
            var_weight = df_distance_weight.loc[var, 'weight']
            var += 1
            if capacity[candidate_index] >= demands[demand_index][2]:
                weight = distance_matrix[candidate_index][demand_index] * var_weight
                if weight < min_weight:
                    min_weight = weight
                    best_candidate = candidate_index
        
        if best_candidate != -1:
            # Check if the total demand capacity of the candidate is within the range [240]
            if  0 <= capacity[best_candidate]-demands[demand_index][2] :
                assignment[demand_index] = best_candidate
                total_weight += weight
                capacity[best_candidate] -= demands[demand_index][2]
                
    
    return assignment, total_weight

# 초기 용량 정보 (용량은 수요지 수용량으로 초기화)
capacity = [candidate[2] for candidate in candidates]

# CFLP 알고리즘 실행
best_assignment, total_weight = solve_cflp(distance_matrix, capacity, demands, df_distance_weight)


# 결과 출력
#for demand_index, candidate_index in enumerate(best_assignment):
#    if candidate_index != -1:
#        demand = demands[demand_index]
#        candidate = candidates[candidate_index]
#        print(f"수요지 {demand}가 후보지 {candidate}에 할당됨")

# 결과 출력
assigned_candidates = [None] * len(candidates)



for demand_index, candidate_index in enumerate(best_assignment):
    if candidate_index != -1:
        if assigned_candidates[candidate_index] is None:
            assigned_candidates[candidate_index] = []
        assigned_candidates[candidate_index].append(demands[demand_index])

# 중복되는 수요지 확인
def check_duplicate_demands(assigned_candidates):
    seen_demands = set()
    duplicate_demands = []

    for demand_list in assigned_candidates:
        if demand_list is not None:
            for demand in demand_list:
                if demand in seen_demands:
                    duplicate_demands.append(demand)
                else:
                    seen_demands.add(demand)

    return duplicate_demands

# 후보지별 할당 결과 출력
for candidate_index, demand_list in enumerate(assigned_candidates):
    if demand_list:
        candidate = candidates[candidate_index]
        unique_demands = set(demand_list)
        demand_names = [f"수요지 {demand[3]}" for demand in unique_demands]
        demands_assigned = ', '.join(demand_names)
        print(f"후보지 {candidate[3]}에 {demands_assigned} 할당됨")
    else:
        print(f"후보지 {candidates[candidate_index][3]}에 할당된 수요지 없음")

# 중복되는 수요지 확인 결과 출력
duplicate_demands = check_duplicate_demands(assigned_candidates)
if duplicate_demands:
    duplicate_demand_names = [f"수요지 {demand[3]}" for demand in duplicate_demands]
    duplicate_names_str = ', '.join(duplicate_demand_names)
    print(f"\n중복되는 수요지: {duplicate_names_str}")
else:
    print("\n중복되는 수요지 없음")

    
    # 각 후보지당 수요지 용량 합, 개수 출력
total_demand_num = 0
for candidate_index, demand_list in enumerate(assigned_candidates):
    if demand_list:
        candidate = candidates[candidate_index]
        total_demand = sum([demand[2] for demand in demand_list])
        total_demand_num += len(set(demand_list))
        print(f"후보지 {candidate[3]}의 수요지 용량 합: {total_demand}")
    else:
        print(f"후보지 {candidates[candidate_index][3]}에 할당된 수요지 없음")
print(total_demand_num)


# df_submission 데이터프레임에 각 수요량 할당
for candidate_index, demand_list in enumerate(assigned_candidates):
    if demand_list:
        candidate_id = candidates[candidate_index][3]
        for demand in demand_list:
            demand_id = demand[3]
            demand_capacity = demand[2]

            condition = (df_submission['수요지 id'] == demand_id) & (df_submission['학교 id'] == candidate_id)
            df_submission.loc[condition, '할당 학령인구'] = demand_capacity

print("총 weight:", total_weight)

후보지 F2에 수요지 C85, 수요지 C78, 수요지 C82, 수요지 C83, 수요지 C86 할당됨
후보지 F3에 수요지 C15, 수요지 C13, 수요지 C14, 수요지 C38, 수요지 C41, 수요지 C44, 수요지 C23, 수요지 C42, 수요지 C29, 수요지 C37, 수요지 C40 할당됨
후보지 F8에 수요지 C280, 수요지 C284, 수요지 C274, 수요지 C275, 수요지 C250, 수요지 C267, 수요지 C281, 수요지 C291, 수요지 C262, 수요지 C286, 수요지 C251, 수요지 C285, 수요지 C279 할당됨
후보지 F10에 수요지 C296, 수요지 C295, 수요지 C306, 수요지 C309, 수요지 C311, 수요지 C305, 수요지 C302, 수요지 C310, 수요지 C299, 수요지 C307 할당됨
후보지 F12에 수요지 C154, 수요지 C130, 수요지 C104, 수요지 C114, 수요지 C109, 수요지 C119, 수요지 C108, 수요지 C124, 수요지 C105, 수요지 C149, 수요지 C126, 수요지 C146, 수요지 C183, 수요지 C95, 수요지 C107, 수요지 C156, 수요지 C111, 수요지 C135, 수요지 C98, 수요지 C113, 수요지 C157, 수요지 C112, 수요지 C141, 수요지 C147, 수요지 C155, 수요지 C94, 수요지 C166, 수요지 C97, 수요지 C96, 수요지 C120, 수요지 C167, 수요지 C148, 수요지 C117, 수요지 C125, 수요지 C103, 수요지 C99, 수요지 C110, 수요지 C140, 수요지 C118 할당됨
후보지 F16에 수요지 C162, 수요지 C201, 수요지 C210, 수요지 C129, 수요지 C214, 수요지 C231, 수요지 C192, 수요지 C220, 수요지 C179, 수요지 C209, 수요지 C223, 수요지 C178, 수요지 C153, 수요지 C180, 수요지 C213, 수요지 C164, 수요지 C221, 수요지 C1

In [5]:
df_submission.head(100)

column_name = '할당 학령인구'  # 여기에 원하는 칼럼의 이름을 넣어주세요
column_sum = df_submission[column_name].sum()
column_sum

10211

In [6]:
# 파일 경로 설정
file_path = r"C:\Users\류석현\Desktop\field\df_result_submission.csv"

# 데이터프레임을 CSV 파일로 저장
df_submission.to_csv(file_path, index=False)

print("데이터프레임이 성공적으로 CSV 파일로 저장되었습니다.")

데이터프레임이 성공적으로 CSV 파일로 저장되었습니다.
